### Library 

In [1]:
import sys
import os
import datetime
import pandas as pd
import numpy as np
import pickle
from matplotlib import pyplot as plt

path_main_folder = '/home/antorosi/Documents/AutoEncoder'
sys.path.append(path_main_folder)

from CVAE.cvae import compile_cvae, run_cvae
from CVAE.cvae_model import CVAE
from conso.load_shape_data import *  

/local/home/antorosi/PycharmProjects/KERAS-TS-VENV/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


### Load and shape data 

In [2]:
# Load raw data
path_data = os.path.join(path_main_folder, 'data')
dict_data_conso = load_data_conso(path_data)

In [3]:
# Unifomization
data_conso_df, dict_colnames_conso = get_uniformed_data_conso(dict_data_conso)

In [4]:
# Change ganularity ?
# data_conso_df = change_granularity(data_conso_df, granularity="1H")

In [5]:
# Get x_conso dataframe for autoencoder purpose
x_conso = get_x_conso_autoencoder(data_conso_df, dict_colnames_conso)

### Build dataset 

In [6]:
# Split train and test
dict_xconso = {'train': x_conso}

In [7]:
# Normalize x_conso
type_scaler = 'standard'
dict_xconso = normalize_xconso(dict_xconso, dict_colnames_conso, type_scaler = 'standard')

In [8]:
# get x and cond for cvae
x, cond, cvae_ds = get_x_cond_autoencoder(dict_xconso['train'])

In [9]:
# create dataset 
dataset = {}
dataset['train'] = {'x': [x, cond], 'y': x}

### Build model

In [10]:
path_out = os.path.join(path_main_folder, 'out')

In [11]:
name_model = 'cvae_conso_e24-12_d24-48_z2_norma-s_v0'

In [12]:
# Parameters
cond_dim = cond.shape[1]
e_dims=[48,24]
d_dims=[24,48]
z_dim=2

In [13]:
model = CVAE(cond_dim=cond_dim, 
             e_dims=e_dims, 
             d_dims=d_dims, 
             z_dim=z_dim, 
             name=name_model, 
             output=path_out)

complete model: 
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
x_true (InputLayer)             (None, 96)           0                                            
__________________________________________________________________________________________________
cond (InputLayer)               (None, 14)           0                                            
__________________________________________________________________________________________________
encoder (Model)                 [(None, 2), (None, 2 6604        x_true[0][0]                     
                                                                 cond[0][0]                       
__________________________________________________________________________________________________
sample_z (Lambda)               (None, 2)            0           encoder[1][0]              

### Trainning model 

In [ ]:
model.main_train(dataset, training_epochs=200, batch_size=20, verbose=True)



--- START TRAINING ---

Epoch 1/200
1830/1830 [==============================] - 0s 136us/step - loss: 39.9962 - kl_loss: 5.7785 - recon_loss: 34.2177
Epoch 2/200
1830/1830 [==============================] - 0s 68us/step - loss: 9.7092 - kl_loss: 3.8937 - recon_loss: 5.8155
Epoch 3/200
1830/1830 [==============================] - 0s 65us/step - loss: 7.2375 - kl_loss: 3.3274 - recon_loss: 3.9102
Epoch 4/200
1830/1830 [==============================] - 0s 64us/step - loss: 6.6087 - kl_loss: 3.1537 - recon_loss: 3.4550
Epoch 5/200
1830/1830 [==============================] - 0s 69us/step - loss: 5.9812 - kl_loss: 2.8933 - recon_loss: 3.0879
Epoch 6/200
1830/1830 [==============================] - 0s 67us/step - loss: 5.6448 - kl_loss: 2.7228 - recon_loss: 2.9219
Epoch 7/200
1830/1830 [==============================] - 0s 65us/step - loss: 5.4566 - kl_loss: 2.6046 - recon_loss: 2.8521
Epoch 8/200
1830/1830 [==============================] - 0s 66us/step - loss: 5.1181 - kl_loss: 2.4875 

1830/1830 [==============================] - 0s 67us/step - loss: 3.1808 - kl_loss: 1.4793 - recon_loss: 1.7015
Epoch 67/200
1830/1830 [==============================] - 0s 71us/step - loss: 3.2411 - kl_loss: 1.4926 - recon_loss: 1.7485
Epoch 68/200
1830/1830 [==============================] - 0s 68us/step - loss: 3.1927 - kl_loss: 1.4521 - recon_loss: 1.7406
Epoch 69/200
1830/1830 [==============================] - 0s 68us/step - loss: 3.1693 - kl_loss: 1.4773 - recon_loss: 1.6919
Epoch 70/200
1830/1830 [==============================] - 0s 69us/step - loss: 3.1571 - kl_loss: 1.4149 - recon_loss: 1.7422
Epoch 71/200
1830/1830 [==============================] - 0s 68us/step - loss: 3.1917 - kl_loss: 1.4678 - recon_loss: 1.7238
Epoch 72/200
1830/1830 [==============================] - 0s 64us/step - loss: 3.1931 - kl_loss: 1.4665 - recon_loss: 1.7266
Epoch 73/200
1830/1830 [==============================] - 0s 68us/step - loss: 3.1583 - kl_loss: 1.4529 - recon_loss: 1.7054
Epoch 74/200


1830/1830 [==============================] - 0s 67us/step - loss: 2.9392 - kl_loss: 1.3623 - recon_loss: 1.5769
Epoch 132/200
1830/1830 [==============================] - 0s 64us/step - loss: 2.9833 - kl_loss: 1.3506 - recon_loss: 1.6327
Epoch 133/200
1830/1830 [==============================] - 0s 73us/step - loss: 2.9632 - kl_loss: 1.3860 - recon_loss: 1.5772
Epoch 134/200
1830/1830 [==============================] - 0s 75us/step - loss: 2.9468 - kl_loss: 1.3517 - recon_loss: 1.5951
Epoch 135/200
1830/1830 [==============================] - 0s 71us/step - loss: 2.9855 - kl_loss: 1.3851 - recon_loss: 1.6003
Epoch 136/200
1830/1830 [==============================] - 0s 71us/step - loss: 2.9587 - kl_loss: 1.3624 - recon_loss: 1.5963
Epoch 137/200
1830/1830 [==============================] - 0s 69us/step - loss: 2.9772 - kl_loss: 1.3977 - recon_loss: 1.5795
Epoch 138/200
1830/1830 [==============================] - 0s 70us/step - loss: 2.9640 - kl_loss: 1.3894 - recon_loss: 1.5746
Epoch 